# ７章  畳み込みニューラルネットワーク

### これまでのシンプルな NN の限界


- アーキテクチャは「Affine - Activate - ... - Affine - Activate - Affine - Final Activate」という並び
- Affine レイヤの入力はベクトル (ランク1テンソル) なので、各ピクセルの輝度値の行列 (カラーならランク3テンソル) を vec で潰してベクトルにして、扱うしかない。
- よって、このシンプルな NN では入力画像データの形状に関する情報 (隣り合うピクセルの輝度値はどれか、RGBのどの輝度値か) を使うことはできない。

### 何が問題か？

- Affine レイヤによって作られる各特徴量は「入力変数を全て使った (全ての入力変数を線形結合した)」ものである。
- その意味で「全結合 (fully-connected) レイヤ」と呼ばれたりもする。
- 学習がとてもうまくいけば「形状情報を考慮した特徴量」が勝手に作られるかもしれない。だが、fully-connected を想定したモデルなので、NN からすると各入力変数の位置関係は分からないので、なかなか難しい。

### じゃあどうするか？

- fully-connected で学習に任せるのではなく、明示的 (強制的) に「位置的に近いピクセルだけをもとに作られる特徴量」をモデルに仮定しておく。
- 例えば、何らかの「画像処理におけるフィルタリングやプーリング」をするレイヤをネットワークに追加すれば、強制的に「形状情報を考慮した特徴量」が作られることになる。
- どういうフィルタリングをするか、という内容についてはパラメータとして学習させる。

平均プーリング：[参考](https://tech-blog.s-yoshiki.com/entry/123)
![fig](https://images-tech-blog.s-yoshiki.com/img/2019/05/20190518004055.png)

ソーベルフィルタ：[参考](https://imagingsolution.net/imaging/filter-algorithm/)
![fig](https://imagingsolution.net/wordpress/wp-content/uploads/2011/03/blog163_10_median7_71.png)![fig](https://imagingsolution.net/wordpress/wp-content/uploads/2011/03/blog163_11.png)


### フィルタリングするための畳み込み層

- CNN では「畳み込み演算」をモデルに仮定 (ネットワークにレイヤ追加) することで、画像処理のフィルタリングを組み込んでいる。
- 演算の詳細は図 7-3, 7-4 がわかりやすい。
- フィルター行列の要素値が「どういうフィルタリングをするのか？」を表す。 CNN ではここはパラメータとして、データから学習させる。
> [p208] CNN の場合、フィルターのパラメータが、これまでの「重み」に対応 します。そして、CNN の場合もバイアスが存在します。
- つまり俗っぽく言うと「職人がこれまで緻密に行ってきたフィルタリングを、AI がデータから自動で行う」みたいなイメージ。特徴量自体を学習する。
- 冷静に考えると、次のような違いがあるだけ。
  - Affine レイヤ：全ての入力変数の線形結合を何個か作る。fully-connected。
  - Conolution レイヤ：隣り合う一部の入力変数だけの線形結合を何個か作る。partially-connected。

### フィルタリング後の画像サイズ

- パディングを作ってやると、フィルタリング後の画像サイズが小さくなりすぎないようにする。図7-6。そうしないと、ディープにしたときに困る。
- 一方、ストライドを大きくすると、フィルタリング後の画像サイズが小さくなる。
- つまり結局、出力画像のサイズは以下の３つで決まる。具体的な計算式は (7.1)。このあたりは基本ハイパラで、学習はさせない。
  - フィルタ行列のサイズ
  - パディングの大きさ
  - ストライドの大きさ



### カラー画像 (複数チャネル画像) のフィルタリング

- カラー＝複数チャネル画像は、数学的にはランク 3 テンソルで表せる。例えば、256 x 256 の RGB 画像なら、サイズ (3, 256, 256) のテンソル。
- CNN の畳み込みでは、入力画像のチャネル数と同じチャネル数のフィルタテンソルを用意して、フィルタリングを行う。図 7-8, 7-9, 7-10。
- 出力画像は1チャネルつまり行列 (ランク2テンソル) になる。特徴マップと呼ぶ。
- 先ほどと同様に「Affine で fully-connected に作っていた線形結合特徴量を、partially-connected に作るように変更しただけ」とも捉えられる。



### 同じ画像に色んなフィルタをかけて色んな特徴取り出す

- [このサイト](https://imagingsolution.net/imaging/filter-algorithm/)からわかるように、色んなフィルタリング方法 (フィルタ行列)があって、全然違う特徴が捉えられる。
- なので、同じ入力画像に複数のフィルタリングを施すようにした方が、多様な特徴を学習できて良いだろう。そこで、ランク 3 フィルタテンソルをたくさん用意して、１個ずつ入力画像に適用して、それぞれで出力される１枚の画像行列を、チャネル方向に重ねて保持する。図 7-11, 7-12。
- ここで「たくさんのランク 3 フィルタテンソル」を「１つのランク 4 フィルタテンソル」として捉えている。

### バッチ処理

- ここまでの説明は全て、１枚の入力画像に対する話。
- 複数枚の画像に対して一気に予測する時や、back propagation で勾配計算を行う時には、バッチ処理したくなる。
- 入力をランク 4 テンソルと見なす。サイズは (バッチサイズ, チャネル数, 横ピクセル数, 縦ピクセル数) となる。図 7-13。
- あとは numpy 等で頑張る。

### プーリング層も使う

- 画像認識 (例えばイヌネコ分類) に使うことをイメージすると分かるように、なるべく「画像のズレ (微小な平行移動) に対してロバスト (値が変わりづらい)」な特徴量が欲しい。
- 学習がとてもうまくいけば、畳み込み層のフィルタで「ズレにロバストな特徴量」が得られる可能性も、一応ある。
- が、基本的に常にこのような特徴量は欲しいので、だったら最初からモデルに組み込んでしまえば良い。
- それがプーリング層。よく使われるのは、Max プーリング。あるピクセル範囲の最大値を取り出すので、明らかに、画像のズレにロバスト。図 7-14, 7-16。

![fig](https://images-tech-blog.s-yoshiki.com/img/2019/05/20190518004109.png)

### Convolution レイヤと Pooling レイヤの比較

- どちらも「隣り合うピクセル群から特徴を取り出す」という意味では同じ。
- Convolution レイヤでは、フィルタ行列の要素値をパラメータとすることで「どのようなフィルタリングが効果的か？」もデータから学習させる。
- Pooling レイヤには、学習するパラメータがない。というか、どういう Pooling　をすべきかをパラメータ化しても良いのだが、事前に「Max プーリングをすればズレにロバストな特徴量が得られて嬉しい」とわかっているので、わざわざ学習させる必要もない。
- Convolution レイヤによるフィルタリング後にはチャネル数を 1 にするが、Pooling レイヤではチャネル数をそのままにする。図 7-8 と 図 7-15 を比較。Pooling レイヤでチャネル数を 1 にまとめてしまうと、カラー画像に対して「ズレにロバストな特徴量」にならないから。

### CNN の全体アーキテクチャ

- 例えば図 7-2。シンプル NN の「Affine - Activate - ... - Affine - Activate - Affine - Final Activate」という並びの前に 「Convolution - Activate - Pooling」というレイヤセットををいくつか追加したもの。
- Pooling を使ったり使わなかったり、細かいバリエーションは色々ある。
- Goodfellow本では「１回でも Convolution してれば CNN」 という風にざっくりと定義している。
> Convolutional networks are simply neural networks that use convolution in place of general matrix multiplication in at least one of their layers.

### CNN の実装

- フルスクラッチで実装するには...
  - back propagation の式を導出。ライプニッツルールをごりごり計算。
  - numpy でテンソルをうまく扱う方法を考えて、計算効率をなるべく下げる。
- いったん諦めて、PyTorch に頼る。

-----

# PyTorch で CNN


### PyTorch のインストール

- ちゃんとやるなら、GPU を用意した環境 (各種クラウド、Google Colab など) で動かすべき。今回はローカルでお試し。
- [PyTorch 公式](https://pytorch.org/get-started/locally/) で OS やパッケージマネージャを選ぶと、インストールコマンドを作ってくれるので、それをターミナルで実行。

In [2]:
# ライブラリの用意

import sys
import os
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from sklearn.metrics import accuracy_score
from torch.utils.data import TensorDataset, DataLoader

sys.path.append(os.pardir)
from dataset.mnist import load_mnist

In [34]:

# 前処理



# MNIST の手書き数字画像データセットを読み込み。

(x_train, y_train), (x_test, y_test) = load_mnist(normalize=True)

print( type(x_train), x_train.shape )
print( type(y_train), y_train.shape )
print( type(x_test), x_test.shape )
print( type(y_test), y_test.shape )



# 学習データ 60000 は多すぎるのでサンプリング

idx = np.random.choice(len(x_train), 20000, replace=False)
x_train = x_train[idx, :]
y_train = y_train[idx]

print( type(x_train), x_train.shape )
print( type(y_train), y_train.shape )



# 入力をランク4テンソル (ndim=4 の numpy 配列) に変換。
#  shape は (サンプルサイズ, チャネル数, 横ピクセル数, 縦ピクセル数)

x_train = x_train.reshape((20000, 1, 28, 28))
x_test = x_test.reshape((10000, 1, 28, 28))

print( type(x_train), x_train.shape, x_train.ndim)
print( type(x_test), x_test.shape, x_test.ndim )



# PyTorch 用のデータ構造に変換。

BATCH_SIZE = 256    # ミニバッチのサイズを設定。

x_train = torch.Tensor(x_train)
y_train = torch.Tensor(y_train).long()
train_ds = TensorDataset(x_train, y_train)
train_dl = DataLoader(dataset=train_ds, batch_size=BATCH_SIZE)

x_test = torch.Tensor(x_test)
y_test = torch.Tensor(y_test).long()
test_ds = TensorDataset(x_test, y_test)
test_dl = DataLoader(dataset=test_ds, batch_size=BATCH_SIZE)

print( type(x_train), x_train.shape, x_train.ndim )
print( type(y_train), y_train.shape, y_train.ndim )
print( type(train_dl) )

<class 'numpy.ndarray'> (60000, 784)
<class 'numpy.ndarray'> (60000,)
<class 'numpy.ndarray'> (10000, 784)
<class 'numpy.ndarray'> (10000,)
<class 'numpy.ndarray'> (20000, 784)
<class 'numpy.ndarray'> (20000,)
<class 'numpy.ndarray'> (20000, 1, 28, 28) 4
<class 'numpy.ndarray'> (10000, 1, 28, 28) 4
<class 'torch.Tensor'> torch.Size([20000, 1, 28, 28]) 4
<class 'torch.Tensor'> torch.Size([20000]) 1
<class 'torch.utils.data.dataloader.DataLoader'>


In [46]:

# CNN モデル (アーキテクチャ) を定義



class MyCNN(nn.Module):
    """ MNIST 用の CNN
    
    お試しで作る CNN なので、特に拡張性とか汎用性は気にしない。
    
    """
    
    def __init__(self):
        """ 初期化
        
        モデルで使うレイヤをここで定義していく。
        torch.nn ライブラリに各種レイヤのテンプレクラスがあるので、そこからインスタンス作る。
        レイヤ単位の forward, backward とかはメソッドとして既に記述してくれているので楽。
        
        """
        
        super().__init__()
        # 継承元の親クラス nn.Module の init を実行。内部の各種設定がされる。
        
        self.conv1 = nn.Conv2d(1, 16, 3)
        # Convolution レイヤ1。
        # チャネル数1の入力画像に、サイズ(3,3)の畳み込みフィルタを、16種類適用。
        # ストライドはデフォルトで 1, パディングはデフォルトで 0。
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros'
        
        self.conv2 = nn.Conv2d(16, 32, 3)
        # Convolution レイヤ2。
        # チャネル数16の入力画像に、サイズ(3,3)の畳み込みフィルタを、32種類適用。
        
        self.pool = nn.MaxPool2d(2, stride=2)
        # Pooling レイヤ。
        # サイズ(2,2)の Max フィルタを、2 ピクセルずつズラしながら適用。
        
        self.ln1 = nn.Linear(32*5*5, 120)
        # Affine レイヤ1。
        # 32*5*5次元入力を、120次元に線形変換。
        # この入力次元は、↓で Conv　レイヤと Pool レイヤの並びを作る時に、算出して調べる。
        
        self.ln2 = nn.Linear(120, 10)
        # Affine レイヤ2。
        # 120 次元入力を、10次元出力に線形変換。
        
        self.ac = nn.ReLU()
        # 活性化関数レイヤ。
        # 今回は ReLU を使ってる。
        
    
    def forward(self, x):
        """ forward propagation
        
        CNN のアーキテクチャ (レイヤの並び) を定義していく。
        
        Args:
          x: 入力。今回の MNIST では 28 x 28 ピクセルで 1 チャネルの画像。
        
        """
        
        x = self.conv1(x)  # 最初に畳み込み
        # 入力： 28 x 28 ピクセルで 1 チャネル
        # 出力： 26 x 26 ピクセルで 16 チャネル
        
        x = self.ac(x)  # 活性化
        # サイズ変化なし
        
        x = self.pool(x)  # Max Pooling
        # 入力： 26 x 26 ピクセルで 16 チャネル
        # 出力： 13 x 13 ピクセルで 16 チャネル
        
        x = self.conv2(x)  # 再び畳み込み
        # 入力： 13 x 13 ピクセルで 16 チャネル
        # 出力： 11 x 11 ピクセルで 32 チャネル
        
        x = self.ac(x)  # 活性化
        # サイズ変化なし
        
        x = self.pool(x)  # 再度 Max Pooling
        # 入力： 11 x 11 ピクセルで 32 チャネル
        # 出力： 5 x 5 ピクセルで 32 チャネル（11番目の行と列ムシされてる）
        
        x = x.view(x.size()[0], -1)  # vec
        # 入力： 5 x 5 ピクセルで 32 チャネル
        # 出力： 5*5*32　次元のベクトル 
        # これ以前の層で形状に関連する特徴量は捉え尽くしたと考え、テンソルをベクトルに潰す。
        
        x = self.ln1(x)  # Affine
        # 入力： 5*5*32　次元のベクトル
        # 出力： 120　次元のベクトル
        
        x = self.ac(x)  # 活性化
        # サイズ変化なし
        
        x = self.ln2(x)  # Affine
        # 入力： 120　次元のベクトル
        # 出力： 10　次元のベクトル  (アウトカムが 10 クラスなので)
        
        return x


In [47]:

# 訓練を実行する関数

def train_fn(model, train_loader, optimizer, epoch):
    
    model.train()
    losses = []
    ys_pred = []
    ys_true = []
    
    for batch_idx, (xs, ys) in enumerate(train_loader):  # ミニバッチのサンプリング
        optimizer.zero_grad()
        hs = model(xs)
        loss = F.cross_entropy(hs, ys)  # ロス関数にはクロスエントロピーを使用。
        loss.backward()
        losses += [loss.item()]

        optimizer.step()
        
        ys_true += ys.tolist()
        ys_pred += hs.argmax(dim=1).tolist()

    return np.mean(losses), accuracy_score(ys_true, ys_pred)



# テストへの予測を実行する関数

def test_fn(model, test_loader, epoch):
    
    model.eval()
    losses = []
    ys_pred = []
    ys_true = []
    with torch.no_grad():
        for batch_idx, (xs, ys) in enumerate(test_loader):
            hs = model(xs)
            loss = F.cross_entropy(hs, ys)
            losses += [loss.item()]
            
            ys_true += ys.tolist()
            ys_pred += hs.argmax(dim=1).tolist()

    return np.mean(losses), accuracy_score(ys_true, ys_pred)



# 「訓練」と「テストデータへの予測評価」を進める関数。

def train_test_fn(model, train_loader, test_loader, optimizer, epoch):
    
    train_loss, train_acc = train_fn(model, train_loader, optimizer, epoch)
    test_loss, test_acc = test_fn(model, test_loader, epoch)
    print(
        f'Epoch: {epoch}',
        f'Train Loss: {np.mean(train_loss):.2f}',
        f'Train Acc: {train_acc}',
        f'Test Loss: {np.mean(test_loss):.2f}',
        f'Test Acc: {test_acc}'
        )



In [52]:

# 訓練 & テストデータ予測評価


model = MyCNN()

for i in range(20):  # ここが epoch 数。
    train_test_fn(
        model, train_dl, test_dl,
        optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.1), epoch=i
    )

# optimizer：Adam
# 学習率： 0.001
# 荷重減衰パラ： 0.1

# 普通の NN (fully-connected しかない NN) との比較とか、まだできていない。
# 訓練サイズを 1/3 にしたわりには、精度 90 % で良い感じだと思う。

Epoch: 0 Train Loss: 1.71 Train Acc: 0.52585 Test Loss: 0.77 Test Acc: 0.773
Epoch: 1 Train Loss: 0.68 Train Acc: 0.8161 Test Loss: 0.57 Test Acc: 0.8483
Epoch: 2 Train Loss: 0.60 Train Acc: 0.83765 Test Loss: 0.53 Test Acc: 0.8645
Epoch: 3 Train Loss: 0.56 Train Acc: 0.8515 Test Loss: 0.50 Test Acc: 0.8704
Epoch: 4 Train Loss: 0.53 Train Acc: 0.85785 Test Loss: 0.48 Test Acc: 0.878
Epoch: 5 Train Loss: 0.50 Train Acc: 0.8702 Test Loss: 0.47 Test Acc: 0.8832
Epoch: 6 Train Loss: 0.49 Train Acc: 0.8733 Test Loss: 0.44 Test Acc: 0.8901
Epoch: 7 Train Loss: 0.47 Train Acc: 0.8823 Test Loss: 0.44 Test Acc: 0.8923
Epoch: 8 Train Loss: 0.46 Train Acc: 0.88495 Test Loss: 0.44 Test Acc: 0.8932
Epoch: 9 Train Loss: 0.46 Train Acc: 0.885 Test Loss: 0.43 Test Acc: 0.8963
Epoch: 10 Train Loss: 0.45 Train Acc: 0.89025 Test Loss: 0.42 Test Acc: 0.9
Epoch: 11 Train Loss: 0.44 Train Acc: 0.89165 Test Loss: 0.42 Test Acc: 0.8979
Epoch: 12 Train Loss: 0.44 Train Acc: 0.89255 Test Loss: 0.41 Test Acc: 0.